# Model Fitting

#### Introduction
Now that we've had a go at running CAMB for different cosmological models and computing/plotting the various obsrevables it computes, we are going to try our hand at fitting some data. In later notebooks we'll use this on CMB power spectra, galaxy power spectra and BAO distances.

However, firstly in this notebook we'll take a step back and look at the fundamentals of model fitting using Bayesian statistics and a technique called Markov-Chain Monte-Carlo (MCMC). This is a class of algorithms designed to generate a probability distribution for the parameters (called the "posterior") we are interested in given our knowledge of the parameters themselves (the "prior") and the a comparison between our data and a model based on those parameters (the "likelihood").

If this is your first time coming across these terms, don't panic! By now you will have run into this already during your studies without realising it; if you've ever fit a line to some points by minimizing the $\chi^{2}$, what you've actually done is compute the likelihood of the model given your data! If you'd included a prior on the parameters that describe this line and worked out some error bars for the slope and intercept, you have really already evaluated the posterior.

By the end of this notebook, you should be able to do the same thing, but using MCMC. We'll get the problem working for a straight line, then move onto some cosmological data in the next workshop.

#### Bayes Theorem
Let's start with a brief overview of Baye's Theorem, the cornerstone of Bayesian Statistics. This theorem can be written (in a simple way that rather understates it's usefulness!) as 
\begin{equation}
P(\theta|D) = \frac{P(D|\theta)P(\theta)}{P(D)}
\end{equation}
where $P(\theta|D)$ is the posterior, the probability of the model parameters $\theta$ given our data, $P(D|\theta)$ is the likelihood of our data given our model with parameters $\theta$, $P(\theta)$ is our prior knowledge of these parameters, and $P(D)$ is the evidence (which we'll ignore from now on).

What this equation says is that if we have a model we think fits our data, and we want to know what the distribution of *parameters* in that model that fits the data (i.e., best-fit parameters and some errors for those), we can do this by computing the likelihood and prior for different sets of parameters. This is the equation that our MCMC algorithm solves.

#### Metropolis-Hastings Algorithm
We are going code up our own algorithm for computing the posterior. We'll start with a simple straight line fit, and then move on to more complicated data later. The algorithm we'll write is a well known and relatively simple one, the Metropolis-Hastings algorithm.

This works as follows (more or less copied from Wikipedia!): First we initialise the algorithm, then we run it iteratively for a fixed number of steps.
* Initialisation:
    1. Choose an arbitrary point to start the algorithm, $\boldsymbol{x}_{0}$.
    2. Choose a proposal distribution. This is the distribution we will use to generate a new set of parameters each iteration based on the current parameters. A Gaussian PDF is usually a good choice as it is symmetric (a requirement for this algorithm) and it means that once we find the best-fit we will explore nearby rather than choosing points far away.
    3. Choose how we are going to evaluate how well the model based on the parameters fits the data (the likelihood). Again a common choice for the likelihood is a Gaussian distribution for the data, which means the log-likelihood is related to the $\chi^{2}$ difference between the data and the model.
    4. Choose the priors we want to place on the parameters we are fitting. If we only want to make sure our parameters remain within some range of values we can make these uniform.
* Iterating: We then start iterating. Each iteration $i$ we:
    1. Choose a new set of values $\boldsymbol{x}'$ from our proposal distribution based on $\boldsymbol{x}_{i}$
    2. Compute the model given the values $\boldsymbol{x}'$
    3. Compute the likelihood of the data given the model.
    4. Compute the ratio of the posteriors for the new values and old values, $\alpha = P(\boldsymbol{x}')/P(\boldsymbol{x}_{i})$
    5. Decide whether or not to accept the new values or stick with the old ones. We do this by generating a uniform random number $u \in [0,1]$ and:
        * If $u \leq \alpha$ we accept the new value and set $\boldsymbol{x}_{i+1}=\boldsymbol{x}'$
        * If $u > \alpha$ we reject the new value and keep $\boldsymbol{x}_{i+1}=\boldsymbol{x}_{i}$
        
With a little thought, you'll see that this algorithm chooses new points in the parameter space somewhat randomly, but will be more likely to stay in regions where the likelihood of the data being generated by the model with those parameters is high. This means that over time it will move towards the best-fit parameters, and then stay there exploring the surrounding regions. Because the probability of moving away from the best-fit is directly proportional to the ratio between the probability at different points in parameter space, the distribution of samples will eventually represent the full likelihood surface for the parameters. Hence by generating histograms of the samples we can work out both the best-fit parameters, and determine error bars for those parameters.

Conceptually, it can be a little difficult to fully appreciate just how this works, so lets just try and implement it

#### Fitting a straight-line
We'll start with an exercise in fitting a straight line. I've set out the framework for the code, but it's up to you to fill in the details

In [11]:
import numpy as np

# This function specifies the priors
def log_prior(parameters):
    
    # Implement some uniform priors for the parameters.
    # ** Add your code here **
    
    # For uniform priors what do we need to return for the log-prior?
    # ** Add your code here **

    
# This function returns the natural logarithm of the likelihood of the data given the model. 
# We assume the data are Gaussian distributed so this is related to the chi_squared value
def log_likelihood(parameters, data_x, data_y, data_err):
    
    # We are fitting a straight line so the model is y = m*x + c, and we want to fit m and c.
    # Compute the chi-squared value between our data and model.
    # ** Add your code here **

    
    # If our likelihood is a Gaussian distribution for each data point, 
    # what do we need to return for the log-likelihood?
    # ** Add your code here **

    
def metropolis_hastings(data_x, data_y, data_err, begin, nsamples, proposal_width):
    
    # Create an array to store the samples
    samples = np.empty((nsamples,len(begin)))
    
    # Evaluate the likelihood for the starting values
    samples[0,0:] = begin
    log_posterior_old = log_prior(samples[0,0:])
    if np.isinf(log_posterior_old):
        print("Starting values are outside your prior range!")
        exit()
    log_posterior_old += log_likelihood(samples[0,0:], data_x, data_y, data_err)

    # Loop over the required number of iterations
    acceptance = 0   # Update this every time you accept a sample
    for i in range(1,nsamples):
        
        # Generate a new set of parameters by drawing from a Gaussian with the proposal width
        # ** Add your code here **
        
        # Compute the log-prior and log-likelihood for the new samples 
        # ** Add your code here **
  
        # Compute the acceptance ratio
        # ** Add your code here **

        # Accept or reject the proposed values and store the results in samples.
        # Update the value of "acceptance" when we accept a sample so we can see how often we do it
        # ** Add your code here **

        
        # Let's print how often we are accepting samples every thousand interations. 
        # This tells us whether we have set reasonable values for the proposal distribution
        if i%10000 == 0:
            print(str("Number of iterations=%d, Acceptance percentage:%d" % (i, int(100.0*acceptance/i))))
        
    # Return the samples for analysis
    return samples
        
# Generate some fake data with Gaussian errors (not all the same though!)
ndata = 30     # The number of data points
mtrue = 2.0    # The true slope of the data
ctrue = 1.0    # The true intercept of the data
data_x = np.linspace(0.0, 1.0, ndata)    # The x values of the data
data_y_true = mtrue*data_x + ctrue       # The true y-values of the data (without measurement error)

# Now generate some errors for your measurements (we have to make sure these are larger than 0)
data_err = np.random.normal(0.5, 0.1, ndata)
index = np.where(data_err <= 0.0)[0]
while len(index) > 0:
    data_err[index] = np.random.normal(0.1, 0.02, len(index))
    index = np.where(data_err <= 0.0)[0]
    
# Now perturb our perfect measurements by the errors. Voila! We have some fake data
data_y = np.random.normal(data_y_true, data_err)

# Now let's run the Metropolis-Hastings algorithm
# Choose some suitable starting values and proposal widths. What do you think would be reasonable values for these?
# We want the proposal width to be large enough that we can hop around the parameter space, but not so large we end
# in areas of low likelihood (far from the best-fit) all the time. One way to tune this is to plot the ratio of
# the samples we accept to the number we have generated. An acceptance percentage of 30-50% is optimal.
mbegin, mproposal =
cbegin, cproposal =
nsamples = int(1e5)
samples = metropolis_hastings(data_x, data_y, data_err, [mbegin, cbegin], nsamples, [mproposal, cproposal])

#### Plotting the results
Once we have a set of samples (or while we are debugging our metropolis-hastings algorithm) we can produce a set of diagnostic and analysis plots of our samples. I've provided some the codes for this below. Because the distribution of the samples should represent the posterior probability distribution of the parameters of our model, we can compute a summary of the parameters by looking at 2D and 1D histograms of the samples. It is also common to quote the peak of the 1D histogram (commonly called the marginalised distribution because we have marginalised out all the other parameters) as the best-fit value. There are several ways for quoting the errors on each value: my way of choice is the upper and lower values that have the same probability and when integrated between contain 68% of the full PDF. A common alternative is to use the 15th, 50th and 84th quantiles. For a gaussian PDF both methods will agree, but this is not generally true.

Fortunately Sam Hinton has written a publicly available python code called ChainConsumer which will analyse the chains, make the various histogram plots and compute the values we want to quote for us.

In [1]:
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer
%matplotlib inline
plt.style.use('ggplot')
plt.style.use('seaborn-deep')
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
font = {'size': 18}
plt.rc('font', **font)

# Now make some plots with the samples. Load in the first 5000 samples and plot them to see if they have converged. 
# Remove the samples generated before the chain has converged to the maximum likleihood (commonly called "burn-in")
c = ChainConsumer()
c.add_chain(samples[0:1000], parameters=[r'$m$',r'$c$'], linewidth=2.0, name="Burn-in")
c.plotter.plot_walks().show()
c.remove_chain(chain="Burn-in")
burnin = 1000
burntin = samples[burnin:]

# Plot the distribution of samples.
c.add_chain(burntin, parameters=[r'$m$',r'$c$'], linewidth=2.0, name="MCMC").configure(summary=True)
c.plotter.plot(figsize="COLUMN", chains="MCMC", truth={'$m$':mtrue,'$c$':ctrue}, display=True)

# Plot the data, best-fit model and 68% confidence interval of models. I'll do this by computing the 
# model values for every sample and them get computing the v16th, 50th and 84th percentiles
model_y = np.outer(data_x, samples[0:,0]) + samples[0:,1]
quantiles = np.quantile(model_y, [0.1587, 0.5, 0.8413], axis=1)

fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
ax.errorbar(data_x, data_y, yerr=data_err, color='k', marker='o', linestyle="None")
ax.plot(data_x, quantiles[1], color='r', linestyle='-', linewidth=1.3)
ax.fill_between(data_x, quantiles[0], quantiles[2], color='k', alpha=0.3)
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$y$')
plt.show()

NameError: name 'samples' is not defined